**From a series of application of modern interpolation methods for economics: written by [Mahdi E Kahou](https://sites.google.com/site/mahdiebrahimikahou/about-me)**


# Goal of this notebook  

The purpose of this notebook is to demonstrate:  
1. How to numerically calculate the expectation of a univariate function $v(x)$ with respect to a normal distribution.  
2. How to apply the Gauss–Hermite quadrature method for this task.  
3. Since these lecture notes focus on machine learning methods, we will emphasize the case where $v(x)$ is represented by a neural network.

 ---

## The Problem

Consider a function $v: \mathbb{R} \to \mathbb{R}$.  
We are interested in numerically computing  

$$
\begin{align*}
\mathbb{E}[v(X)] = \int_{-\infty}^{\infty} v(x) f(x;\mu,\sigma) dx
\end{align*}
$$

where $f(x;\mu,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$ is the pdf of a normal distribution $\mathcal{N}(\mu,\sigma^2)$.

---